<a href="https://colab.research.google.com/github/daniel27a/missp/blob/main/pulp/Whiskas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook zawiera rozwiązanie problemu liniowego przy pomocy narzędzia Pulp.

Poniższa komórka tekstu zawiera instrukcję instalującą dodatek Pulp do Notebooka. 


In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

In [ ]:
"""
The Simplified Whiskas Model Python Formulation for the PuLP Modeller



"""

from pulp import *

prob = LpProblem("The Whiskas' Cat Food Problem",LpMinimize)

# The 2 variables Beef and Chicken are created with a lower limit of zero
x1=LpVariable("ChickenPercent",0,)
x2=LpVariable("BeefPercent",0)
x3=LpVariable("SardinesPercent",0)

# The objective function is added to 'prob' first
prob += 0.013*x1 + 0.008*x2 + 0.01*x3, "Total Cost of Ingredients per can"

# The five constraints are entered
prob += x1 + x2 + x3 == 100, "PercentagesSum"
prob += 0.100*x1 + 0.200*x2 + 0.25*x3 >= 8.0, "ProteinRequirement"
prob += 0.080*x1 + 0.100*x2 + 0.11*x3 >= 6.0, "FatRequirement"
prob += 0.001*x1 + 0.005*x2 + 0*x3 <= 2.0, "FibreRequirement"
prob += 0.002*x1 + 0.005*x2 + 0.00505*x3 <= 0.4, "SaltRequirement"
prob += 0.003*x1 + 0.026*x2 + 0.089*x3 >= 0.8, "VitaminBreq" 

prob.writeLP("WhiskasModel.lp")

prob.solve()
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

produkty_procentowo = ""

for v in prob.variables():
    produkty_procentowo += v.name + " = " + str(v.varValue)+ "%, " 

produkty_procentowo = produkty_procentowo[:-2]

print("Puszka produktu będzie kosztować ", value(prob.objective), "$ i będzie miał w składzie:", produkty_procentowo,".")

print("-"*30)